<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training5/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("gdrive")

Mounted at gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/translation2019zh.zip

Archive:  /content/gdrive/MyDrive/translation2019zh.zip
  inflating: translation2019zh_train.json  
  inflating: translation2019zh_valid.json  


# read json

In [23]:
!pip install opencc

     |████████████████████████████████| 765 kB 36.0 MB/s 


In [3]:
import json
import opencc
train_data = []

chineseCc = opencc.OpenCC('s2hk.json')
with open("translation2019zh_train.json",newline="") as js:
  for i in js:
    data = json.loads(i)
    data["english"] = chineseCc.convert(data["english"])
    data["chinese"] = chineseCc.convert(data["chinese"])
    train_data.append(data)
  js.close()

In [6]:
train_input = []
train_output = []
input_chars = set()
ouput_chars = set()

for data in train_data:
  input = data["english"]
  output = data["chinese"] #add \t for begin and \n for the end
  output = "\t" + output + "\n"
  train_input.append(input)
  train_output.append(output)
  #save each character to a set
  for char in input: # english material is include Chinese！？？
    if char not in input_chars:
      input_chars.add(char)

  for char in output:
    if char not in ouput_chars:
      ouput_chars.add(char)

In [20]:
input_chars = sorted(list(input_chars))
ouput_chars = sorted(list(ouput_chars))

#number of encoder and decoder
num_encoder_tk = len(input_chars)
num_decoder_tk = len(ouput_chars)

#max encoder and decoder length
max_encoder_length = max([len(txt) for txt in train_input]) #get the maxinum txt in input list
max_decoder_length = max([len(txt) for txt in train_output]) #get the maxinum txt in output list

# Encoder

In [ ]:
#mapping tk and index
import numpy as np
from keras.models import Model
from keras.layers import Input,LSTM,Dense

input_tk_index = dict([(char,i) for i,char in enumerate(input_chars)])
output_tk_index = dict([(char,i) for i,char in enumerate(ouput_chars)])

#make shape using np.zero as 3d shape
encoder_input_dt = np.zeros((len(train_input),max_encoder_length,num_encoder_tk),dtype="float32")
decoder_input_dt = np.zeros((len(train_input),max_decoder_length,num_decoder_tk),dtype="float32")
decoder_output_dt = np.zeros((len(train_input),max_decoder_length,num_decoder_tk),dtype="float32")

#change to vector 3d
for i ,(train_input,train_output) in enumerate(zip(train_input,train_output)):
  for t,char in enumerate(train_input):
    #only z-index has value = 1
    encoder_input_dt[i,t,input_tk_index[char]]  = 1
  for t,char in enumerate(train_output):
    #only z-index has value = 1
    decoder_input_dt[i,t,output_tk_index[char]]  = 1
    if t > 0:
      decoder_output_dt[i,t-1,output_tk_index[char]] = 1
  


In [ ]:
#content
encoder_inputs = Input(shape=(None,num_encoder_tk))

#return last state output as decoder input
encoder_lstm = LSTM((latent_dim),return_state=True)

#lstm return output
encoder_output,state_history,state_current = encoder_lstm(encoder_inputs)

#keep the state only
encoder_states = [state_history,state_current]

# Decoder

In [ ]:
decoder_inputs = Input(shape=(None,num_decoder_tk))
decoder_lstm = LSTM((latent_dim),return_sequences=True,return_state=True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

#full connected layer
decoder_dense = Dense(num_decoder_tk,activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

#Define a model
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [ ]:
#compile the model
model.compile(optimizer="rmsprop",loss="categorical_crossentropy")

#model optimizsm
model.fit([encoder_input_dt,decoder_input_dt],decoder_output_dt,batch_size=64,epochs=100,validation_split=0.2)

#save model
model.save("seq2seq.h5")